In [1]:
import sys

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

The sys.setrecursionlimit function is used to increase the recursion limit, which helps prevent potential recursion errors when running complex models with deep nested functions or when using certain libraries like TensorFlow.

- **`keras_tuner`**: Used for hyperparameter tuning.
- **`Sequential`**: A linear stack of layers in Keras.
- **`Dense`**, **`Flatten`**: Common Keras layers.
- **`mnist`**: The MNIST dataset, a standard dataset for image classification.
- **`Adam`**: An optimizer in Keras.

In [2]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only

- **`mnist.load_data()`**: Loads the dataset, returning training and validation splits.
- **`x_train / 255.0`**: Normalizes the pixel values to be between 0 and 1.
- **`print(f'...')`**: Displays the shapes of the training and validation datasets.

In [6]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


### Define the model with Hyperparameters
**Define a model-building function:**
- Create a function `build_model` that takes a `HyperParameters` object as input.
- Use the `HyperParameters` object to define the number of units in a dense layer and the learning rate for the optimizer.
- Compile the model with sparse categorical cross-entropy loss and Adam optimizer.


- **`hp.Int('units', ...)`**: Defines the number of units in the Dense layer as a hyperparameter.
- **`hp.Float('learning_rate', ...)`**: Defines the learning rate as a hyperparameter.
- **`model.compile()`**: Compiles the model with the Adam optimizer and sparse categorical cross-entropy loss.


In [7]:
# Define a model-building function 

def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


### Create a RandomSearch Tuner:

Use the RandomSearch class from Keras Tuner.
Specify the model-building function, optimization objective (validation accuracy), number of trials, and directory for storing results.

- **`build_model`**: The model-building function.
- **`objective='val_accuracy'`**: The metric to optimize (validation accuracy).
- **`max_trials=10`**: The maximum number of different hyperparameter configurations to try.
- **`executions_per_trial=2`**: The number of times to run each configuration.
- **`directory='my_dir'`**: Directory to save the results.
- **`project_name='intro_to_kt'`**: Name of the project for organizing results.

In [8]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space 
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


- **`epochs=5`**: Each trial is trained for 5 epochs.
- **`validation_data=(x_val, y_val)`**: The validation data to evaluate the model's performance during the search.

In [9]:
# Run the hyperparameter search 
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

# Display a summary of the results 
tuner.results_summary() 

Trial 10 Complete [00h 00m 14s]
val_accuracy: 0.9611499905586243

Best val_accuracy So Far: 0.9784500002861023
Total elapsed time: 00h 05m 19s
Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 08 summary
Hyperparameters:
units: 224
learning_rate: 0.0007784635441963913
Score: 0.9784500002861023

Trial 07 summary
Hyperparameters:
units: 256
learning_rate: 0.0024479132350219944
Score: 0.9767000079154968

Trial 02 summary
Hyperparameters:
units: 416
learning_rate: 0.0040812671360801476
Score: 0.9766499996185303

Trial 01 summary
Hyperparameters:
units: 128
learning_rate: 0.0012179247857147329
Score: 0.9764499962329865

Trial 05 summary
Hyperparameters:
units: 416
learning_rate: 0.003863170317069186
Score: 0.9727999866008759

Trial 04 summary
Hyperparameters:
units: 384
learning_rate: 0.0001383021456134251
Score: 0.9700500071048737

Trial 03 summary
Hyperparameters:
units: 480
learning_rate: 0.006822989239763269
Score

In [10]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(f""" 

The optimal number of units in the first dense layer is {best_hps.get('units')}. 

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 

""") 

# Step 2: Build and Train the Model with Best Hyperparameters 
model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

# Evaluate the model on the test set 
test_loss, test_acc = model.evaluate(x_val, y_val) 
print(f'Test accuracy: {test_acc}') 

 

The optimal number of units in the first dense layer is 224. 

The optimal learning rate for the optimizer is 0.0007784635441963913. 


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8671 - loss: 0.4725 - val_accuracy: 0.9574 - val_loss: 0.1532
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9638 - loss: 0.1250 - val_accuracy: 0.9661 - val_loss: 0.1114
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9769 - loss: 0.0788 - val_accuracy: 0.9712 - val_loss: 0.0955
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9842 - loss: 0.0549 - val_accuracy: 0.9741 - val_loss: 0.0856
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9880 - loss: 0.0403 - val_accuracy: 0.9722 - val_loss: 0.0877
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9916 - loss: 0.0295 - val_accuracy: 0.9771 - val_loss: 0.0821
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9942 - loss: 